In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import Levenshtein as lev
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-f6808bf80ee8.json') #credenciales
project_id = 'tidy-hold-359719' #nombre del proyecto
client = bigquery.Client(credentials= credentials,project=project_id) #creo cliente
dataset_ref = client.dataset("comision_latinoamericana", project=project_id) #referencia al dataset
dataset = client.get_dataset(dataset_ref) #dataset
#QUERYS
q_d_o="""SELECT * FROM `tidy-hold-359719.comision_latinoamericana.datos_ONU`"""   #query para tabla NDC
d_o = client.query(q_d_o).to_dataframe() #dataframe de NDC

In [3]:
d_o.sort_values(['Pais','Anio'])

,Pais,Anio,intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI,proporcion_de_la_poblacion_con_acceso_a_elecricidad,proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias,proporcion_de_energias_renovables_del_total_consumido
400,Antigua y Barbuda,2000,3.13,97.68926,96,0.00
401,Antigua y Barbuda,2001,3.31,97.78526,96,0.00
402,Antigua y Barbuda,2002,3.59,100.00000,96,0.00
403,Antigua y Barbuda,2003,3.63,97.95683,96,0.00
404,Antigua y Barbuda,2004,3.57,98.03710,96,0.00
...,...,...,...,...,...,...
395,Uruguay,2015,2.75,99.70948,96,59.40
396,Uruguay,2016,2.82,99.70000,96,60.32
397,Uruguay,2017,2.71,99.80000,96,60.82
398,Uruguay,2018,2.79,99.80000,96,60.86


In [4]:
d_o.to_csv('datos_ONU.csv', index=False)

In [4]:
def extraer_tablas():
  credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-f6808bf80ee8.json') #credenciales
  project_id = 'tidy-hold-359719' #nombre del proyecto
  client = bigquery.Client(credentials= credentials,project=project_id) #creo cliente
  dataset_ref = client.dataset("proyecto", project=project_id) #referencia al dataset
  dataset = client.get_dataset(dataset_ref) #dataset
  #QUERYS
  q_NDC="""SELECT * FROM `tidy-hold-359719.proyecto.NDC`"""   #query para tabla NDC
  
  q_datos_ONU = """                              
    SELECT a.GeoAreaName as Pais,
    a.TimePeriod as Anio,
    a.Value as intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI,
    b.Value as proporcion_de_la_poblacion_con_acceso_a_elecricidad,
    c.Value as proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias,
    d.Value as proporcion_de_energias_renovables_del_total_consumido,
    FROM `tidy-hold-359719.proyecto.energy_intensity_measured_in_terms_of_primary_energy_and_GDP` a
    LEFT JOIN `tidy-hold-359719.proyecto.proportion_of_population_with_access_to_electricity` b ON (a.GeoAreaName=b.GeoAreaName and a.TimePeriod = b.TimePeriod) AND b.Location='ALLAREA'
    LEFT JOIN `tidy-hold-359719.proyecto.proportion_of_population_with_primary_reliance_on_clean_fuels_and_technology` c ON (a.GeoAreaName=c.GeoAreaName and a.TimePeriod = c.TimePeriod)
    LEFT JOIN `tidy-hold-359719.proyecto.renewable_energy_share_in_the_total_final_energy_consumption` d ON (a.GeoAreaName=d.GeoAreaName and a.TimePeriod = d.TimePeriod)
    """
  q_energyco2 = """
    SELECT Country AS Pais, Year AS Anio, CO2_emission AS Emisiones_de_CO2
    FROM `tidy-hold-359719.proyecto.energyco2`
    WHERE Energy_type = 'all_energy_types'
    """
  
  q_Temperaturas = """
    SELECT * FROM `tidy-hold-359719.proyecto.temperatures`
    """
  NDC = client.query(q_NDC).to_dataframe() #dataframe de NDC
  datos_ONU = client.query(q_datos_ONU).to_dataframe() #dataframe de datos onu
  energyco2 = client.query(q_energyco2).to_dataframe() 
  temperaturas = client.query(q_Temperaturas).to_dataframe()
  return NDC,datos_ONU,energyco2,temperaturas

def traductor(df):
  for i in range(len(df)):
    for j in range(len(NDC)):
        if lev.ratio(df.Pais[i], NDC.Pais[j]) > 0.6:
            df.Pais[i]= NDC.Pais[j]
  df.loc[df.Pais=='World', 'Pais']='Mundo'
  df.loc[df.Pais=='United States', 'Pais']='Estados Unidos'
  df.loc[df.Pais=='Dominican Republic', 'Pais']='República Dominicana'
  df.loc[df.Pais=='Saint Kitts and Nevis', 'Pais']='San Cristóbal y Nieves'
  return df

def filtro_latam(df):
  df = df[df.Pais.isin(NDC.Pais.unique().tolist())]
  df.reset_index(drop=True, inplace=True)
  return df

def carga(df):
  credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-f6808bf80ee8.json') #credenciales
  project_id = 'tidy-hold-359719' #nombre del proyecto
  client = bigquery.Client(credentials= credentials,project=project_id) #creo cliente
  table_id = "tidy-hold-359719.comision_latinoamericana."+[ k for k,v in globals().items() if v is df][0]
  #configuracion de la carga
  job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
  job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  
  # Make an API request.
  job.result()  # Wait for the job to complete.
  table = client.get_table(table_id)  # Make an API request.
  print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))

def nans(df):
  df.fillna(-1,inplace=True) #nan con -1 para despues filtrarlos
def erroneos(df):
  df.replace({'>95':96,'NaN':-1,'<5':4},inplace=True)

NDC, datos_ONU, energyco2,temperaturas = extraer_tablas()

In [5]:
datos_ONU.sort_values(['Pais','Anio'])[datos_ONU.sort_values(['Pais','Anio'])['Pais'] == 'Uruguay']

,Pais,Anio,intensidad_energetica_medida_en_terminos_de_energia_primaria_y_PBI,proporcion_de_la_poblacion_con_acceso_a_elecricidad,proporcion_de_la_poblacion_con_dependencia_primaria_a_energias_limpias,proporcion_de_energias_renovables_del_total_consumido
2655,Uruguay,2000,2.70,97.67686,>95,38.73
2656,Uruguay,2001,2.47,97.78539,>95,41.63
2657,Uruguay,2002,2.48,97.88633,>95,43.25
2658,Uruguay,2003,2.48,97.98204,>95,43.83
2659,Uruguay,2004,2.69,98.07486,>95,39.31
2660,Uruguay,2005,2.60,98.16714,>95,41.32
2661,Uruguay,2006,2.70,98.50543,>95,37.60
2662,Uruguay,2007,2.53,98.69264,>95,43.20
2663,Uruguay,2008,3.05,98.78436,>95,44.09
2664,Uruguay,2009,2.89,99.10000,>95,44.64


In [7]:
nans(datos_ONU)
erroneos(datos_ONU)
carga(datos_ONU)

ArrowTypeError: Expected bytes, got a 'int' object